In [2]:
import syft as sy
import pandas as pd
import numpy as np
from syft.core.adp.entity import DataSubject
import logging

# third party
from nacl.encoding import HexEncoder
from nacl.signing import SigningKey
import numpy as np
import pytest

# syft absolute
import syft as sy
from syft import Domain
from syft.core.adp.adversarial_accountant import AdversarialAccountant
from syft.core.adp.entity import DataSubject
from syft.core.adp.scalar import PhiScalar
from syft.core.node.common.node_table.utils import create_memory_db_engine
from syft.core.tensor.tensor import Tensor

Seeding DB with roles!


In [3]:
data = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv")

/Users/andrewliamtrask/opt/anaconda3/envs/syft/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
canada = sy.login(port=8081)

No email/password specified. Logging in with default...
Don't forget to re-configure your admin email and password!!!
Connecting to http://localhost:8081... done! 	 Logging into canada... done!


In [5]:
# This is the initial privacy budget set for the admin user
canada.privacy_budget

55.55

# Step 1: Create our private dataset

In [6]:
# Get canada trade data and convert into unit millions
canada_trade = (np.array(list(data['Trade Value (US$)'])) / 1000000)[0:10]
trade_partners = ((list(data['Partner'])))[0:10]

In [7]:
data_batch = canada_trade

entities = list()
for i in range(len(data_batch)):
    entities.append(DataSubject(name=trade_partners[i]))

# Upload a private dataset to the Domain object, as the root owner
private_data = sy.Tensor(data_batch).private(0, 2, entities=entities).tag("trade_flow")

# Step 2: Create a local Domain and publish

In [8]:

db_engine, _ = create_memory_db_engine()
domain = Domain(name="Bob", db_engine=db_engine)

def encode_key(key: SigningKey) -> str:
    return key.encode(encoder=HexEncoder).decode("utf-8")

key = SigningKey.generate()
domain.users.signup(
    name="Bob",
    email="bob@gmail.com",
    password="letmein",
    budget=55,
    role=1,
    private_key=encode_key(key),
    verify_key=encode_key(key.verify_key),
)

Seeding DB with roles!
Setup hasnt run yet so ignoring set_node_uid


In [9]:
g = private_data.gamma # you have to do this because only gamma tensors can be published
out = g.publish(acc=domain.acc, sigma=80, user_key=key.verify_key)

IntermediateGamma.publish
<class 'syft.core.tensor.autodp.initial_gamma.InitialGammaTensor'>
<class 'list'>


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


Has Budget:True
SPEND:0.33493329505335045
USER BUDGET:55.0


/Users/andrewliamtrask/opt/anaconda3/envs/syft/lib/python3.8/site-packages/scipy/optimize/optimize.py:2555: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


In [10]:
domain.acc.print_ledger(returned_epsilon_is_private=False)

Other Asia, nes	0.33493329505335045
Singapore	0.33493329505335045
South Africa	0.33493329505335045
Sweden	0.33493329505335045
United Kingdom	0.33493329505335045
Egypt	0.33493329505335045
Viet Nam	0.33493329505335045
Spain	0.33493329505335045
Venezuela	0.33493329505335045
United Rep. of Tanzania	0.33493329505335045


In [11]:
domain.acc.get_remaining_budget(user_key=key.verify_key)

54.665066704946646

# Step 3: Send data to remote Domain, publish and compare results

In [12]:
# Send data to the domain object
data_ptr = private_data.send(canada)

In [14]:
# canada.store

In [15]:
# out = data_ptr.sum(0)
g = data_ptr.gamma

In [16]:
public_out2 = g.publish(client=canada, sigma=80)

a
b
c
d


In [17]:
# public_out2.get()

In [18]:
assert canada.privacy_budget == domain.acc.get_remaining_budget(user_key=key.verify_key)

AssertionError: 

In [19]:
canada.privacy_budget

55.21506670494664

In [63]:

canada.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget":10
    }
)


[2021-08-26T01:32:52.321757+0100][INFO][user_api]][8139] User created successfully!


In [64]:
sheldon = sy.login(email="sheldon@caltech.edu",
                   password="bazinga",
                   port=8081)

Connecting to http://localhost:8081...

[2021-08-26T01:32:53.193227+0100][INFO][logger]][8139] Skipping torch.Tensor.fft not supported in 1.8.1
[2021-08-26T01:32:53.193996+0100][INFO][logger]][8139] Skipping torch.Tensor.ifft not supported in 1.8.1
[2021-08-26T01:32:53.194536+0100][INFO][logger]][8139] Skipping torch.Tensor.irfft not supported in 1.8.1
[2021-08-26T01:32:53.195122+0100][INFO][logger]][8139] Skipping torch.Tensor.rfft not supported in 1.8.1
[2021-08-26T01:32:53.198816+0100][INFO][logger]][8139] Skipping torch.ifft not supported in 1.8.1
[2021-08-26T01:32:53.199631+0100][INFO][logger]][8139] Skipping torch.irfft not supported in 1.8.1
[2021-08-26T01:32:53.200205+0100][INFO][logger]][8139] Skipping torch.rfft not supported in 1.8.1


 done! 	 Logging into canada... done!


In [65]:
sheldon.privacy_budget

10.0

In [66]:
ptr = sheldon.store[0]

In [71]:
out = ptr.sum(0).publish(client=sheldon, sigma=0.1)

a
b
c
d


In [73]:
# out.get()

In [70]:
# y = ptr + ptr

In [69]:
# y.get()

In [44]:
type(ptr)

syft.proxy.syft.core.tensor.tensor.TensorPointer

In [41]:
out.get()

[2021-08-26T01:04:58.361988+0100][ERROR][logger]][8139] UnknownPrivateException has been triggered.
[2021-08-26T01:04:58.362876+0100][CRITICAL][logger]][8139] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.